In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
# Use Zipcode (and State) as baseline for everything?

# Make documentation of all endpoints too
# Create some sample user requests from census data

# American Community Survey
# All house based data with 3.13 people per household
# https://data.census.gov/table?g=0100000US$8600000&tid=ACSST5Y2021.S2502 = demographics
# https://data.census.gov/table?g=0100000US$8600000&tid=ACSDT5Y2021.B01001 = sex/age

### Demographic Breakdown
#### Understand the Demographic breakdown of a Zipcode or a State <br /><br /> Endpoint: /demographics?

In [3]:
# Read in metadata
demographicsmetadf = pd.read_csv('Demographics2021Metadata.csv')
demographicsmetadf.describe()

,Column Name,Label
count,650,650
unique,650,650
top,GEO_ID,Geography
freq,1,1


In [4]:
# Read in data
demographicsdf = pd.read_csv('Demographics2021.csv', skiprows=1)
demographicsdf = demographicsdf.drop(['Unnamed: 650'], axis=1)
demographicsdf.head()

,Geography,Geographic Area Name,Estimate!!Occupied housing units!!Occupied housing units,Margin of Error!!Occupied housing units!!Occupied housing units,Annotation of Margin of Error!!Occupied housing units!!Occupied housing units,Annotation of Estimate!!Occupied housing units!!Occupied housing units,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!White,Margin of Error!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!White,Annotation of Margin of Error!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!White,Annotation of Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!White,...,Margin of Error!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 2000 to 2009,Annotation of Margin of Error!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 2000 to 2009,Estimate!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1990 to 1999,Annotation of Estimate!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1990 to 1999,Margin of Error!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1990 to 1999,Annotation of Margin of Error!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1990 to 1999,Estimate!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1989 or earlier,Margin of Error!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1989 or earlier,Annotation of Margin of Error!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1989 or earlier,Annotation of Estimate!!Percent renter-occupied housing units!!Occupied housing units!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Moved in 1989 or earlier
0,860Z200US00601,ZCTA5 00601,5397,264,NaN,NaN,4640,253,NaN,NaN,...,4.4,NaN,9.2,NaN,4.6,NaN,18.7,6.3,NaN,NaN
1,860Z200US00602,ZCTA5 00602,12858,448,NaN,NaN,7735,418,NaN,NaN,...,5.6,NaN,9.1,NaN,3.8,NaN,20.1,6.5,NaN,NaN
2,860Z200US00603,ZCTA5 00603,19295,555,NaN,NaN,14013,613,NaN,NaN,...,3.0,NaN,10.0,NaN,2.6,NaN,15.6,3.1,NaN,NaN
3,860Z200US00606,ZCTA5 00606,1968,171,NaN,NaN,1053,154,NaN,NaN,...,9.0,NaN,20.8,NaN,11.4,NaN,6.6,6.4,NaN,NaN
4,860Z200US00610,ZCTA5 00610,8934,372,NaN,NaN,4907,425,NaN,NaN,...,5.0,NaN,9.1,NaN,3.9,NaN,21.8,6.6,NaN,NaN


In [5]:
# remove unwanted data columns
demcolumns = demographicsdf.columns.values
demcolumns = list(demcolumns)
errcolumns = [col for col in demcolumns if "Error!!" in col]
print(str(len(errcolumns)) + ' out of ' + str(len(demcolumns)) + ' removed for being error columns')
demcolumns = [col for col in demcolumns if "Error!!" not in col]
anncolumns = [col for col in demcolumns if "Annotation" in col]
print(str(len(anncolumns)) + ' out of ' + str(len(demcolumns)) + ' removed for being annotated columns')
demcolumns = [col for col in demcolumns if "Annotation" not in col]
print(str(len(demcolumns)) + ' columns remaining')

324 out of 650 removed for being error columns
162 out of 326 removed for being annotated columns
164 columns remaining


In [6]:
# select relevant data columns
demcolumns = demcolumns[:23]
demographicsdf = demographicsdf[demcolumns]
demographicsdf.describe()

,Estimate!!Occupied housing units!!Occupied housing units,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!White,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!Black or African American,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!American Indian and Alaska Native,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!Asian,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!Native Hawaiian and Other Pacific Islander,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!One race --!!Some other race,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Two or more races,Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Hispanic or Latino origin,"Estimate!!Occupied housing units!!Occupied housing units!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!White alone, not Hispanic or Latino",...,Estimate!!Occupied housing units!!Occupied housing units!!AGE OF HOUSEHOLDER!!35 to 44 years,Estimate!!Occupied housing units!!Occupied housing units!!AGE OF HOUSEHOLDER!!45 to 54 years,Estimate!!Occupied housing units!!Occupied housing units!!AGE OF HOUSEHOLDER!!55 to 64 years,Estimate!!Occupied housing units!!Occupied housing units!!AGE OF HOUSEHOLDER!!65 to 74 years,Estimate!!Occupied housing units!!Occupied housing units!!AGE OF HOUSEHOLDER!!75 to 84 years,Estimate!!Occupied housing units!!Occupied housing units!!AGE OF HOUSEHOLDER!!85 years and over,Estimate!!Occupied housing units!!Occupied housing units!!EDUCATIONAL ATTAINMENT OF HOUSEHOLDER!!Less than high school graduate,Estimate!!Occupied housing units!!Occupied housing units!!EDUCATIONAL ATTAINMENT OF HOUSEHOLDER!!High school graduate (includes equivalency),Estimate!!Occupied housing units!!Occupied housing units!!EDUCATIONAL ATTAINMENT OF HOUSEHOLDER!!Some college or associate's degree,Estimate!!Occupied housing units!!Occupied housing units!!EDUCATIONAL ATTAINMENT OF HOUSEHOLDER!!Bachelor's degree or higher
count,33774.000000,33774.000000,33774.000000,33774.000000,33774.000000,33774.000000,33774.000000,33774.000000,33774.000000,33774.000000,...,33774.000000,33774.000000,33774.000000,33774.000000,33774.00000,33774.000000,33774.000000,33774.000000,33774.000000,33774.000000
mean,3707.204654,2690.089803,454.281311,25.744508,182.081512,5.001421,158.484011,191.522088,542.902973,2425.526381,...,640.948777,671.306508,728.145408,572.981051,282.18304,119.276485,348.751170,881.365755,1129.896873,1347.190857
std,5469.194574,3868.248897,1456.982534,104.636986,704.756698,47.618967,596.802562,451.166042,1733.045664,3545.708538,...,1049.485685,1036.819153,1040.129193,813.055095,427.88297,203.704144,736.647414,1280.949130,1699.235354,2489.698423
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,240.000000,205.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,190.000000,...,28.000000,37.000000,52.000000,44.000000,20.00000,4.000000,16.000000,79.000000,74.000000,42.000000
50%,1002.500000,842.000000,3.000000,0.000000,0.000000,0.000000,1.000000,21.000000,21.000000,772.000000,...,142.500000,173.000000,221.000000,185.000000,89.00000,30.000000,81.000000,311.000000,319.000000,212.000000
75%,5032.000000,3656.750000,196.000000,13.000000,53.000000,0.000000,62.000000,159.000000,245.000000,3172.000000,...,794.000000,887.000000,1017.000000,808.000000,381.00000,145.000000,342.000000,1161.750000,1482.000000,1423.750000
max,41418.00

In [7]:
# rename relevant data columns
newdemcols = demcolumns[:2]
for col in demcolumns:
    if '!!' in col:
        newdemcols.append(col.split("!!")[-1])
demographicsdf.columns = newdemcols
demographicsdf.head()

,Geography,Geographic Area Name,Occupied housing units,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,Two or more races,...,35 to 44 years,45 to 54 years,55 to 64 years,65 to 74 years,75 to 84 years,85 years and over,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher
0,860Z200US00601,ZCTA5 00601,5397,4640,28,4,0,0,500,225,...,768,998,1242,1065,600,189,1685,1645,1050,1017
1,860Z200US00602,ZCTA5 00602,12858,7735,222,12,0,0,501,4388,...,1585,2233,3065,2926,1669,558,4748,3324,2381,2405
2,860Z200US00603,ZCTA5 00603,19295,14013,883,26,19,0,2836,1518,...,2709,3105,4029,4011,2525,608,5427,5478,3648,4742
3,860Z200US00606,ZCTA5 00606,1968,1053,0,11,0,0,793,111,...,236,427,363,410,219,152,851,598,363,156
4,860Z200US00610,ZCTA5 00610,8934,4907,198,0,0,0,571,3258,...,1123,1659,2016,1793,1106,277,2750,2607,1899,1678


In [8]:
# isolate zip code
demographicsdf = demographicsdf.rename({'Geography': 'Zip Code'}, axis=1)
demographicsdf = demographicsdf.drop(['Geographic Area Name'], axis=1)
demographicsdf['Zip Code'] = demographicsdf['Zip Code'].str[-5:]
demographicsdf.head()

,Zip Code,Occupied housing units,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,Two or more races,Hispanic or Latino origin,...,35 to 44 years,45 to 54 years,55 to 64 years,65 to 74 years,75 to 84 years,85 years and over,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher
0,00601,5397,4640,28,4,0,0,500,225,5364,...,768,998,1242,1065,600,189,1685,1645,1050,1017
1,00602,12858,7735,222,12,0,0,501,4388,12385,...,1585,2233,3065,2926,1669,558,4748,3324,2381,2405
2,00603,19295,14013,883,26,19,0,2836,1518,18982,...,2709,3105,4029,4011,2525,608,5427,5478,3648,4742
3,00606,1968,1053,0,11,0,0,793,111,1962,...,236,427,363,410,219,152,851,598,363,156
4,00610,8934,4907,198,0,0,0,571,3258,8658,...,1123,1659,2016,1793,1106,277,2750,2607,1899,1678


In [9]:
# Save data
demographicsdf.to_csv('CleanedDemographics2021.csv', index=False)

In [10]:
# Convert housing to population?
# multiply everything by 3.13 people per house

In [11]:
# Sample endpoint to emulate endpoint
def demographics(zipCode):
    zipInfodf = getZipInfo(zipCode)
    zipInfo = formatZipData(zipInfodf)
    return(zipInfo)

In [12]:
# helper method to retrieve data
def getZipInfo(zipCode):
    demographicsdf = pd.read_csv('CleanedDemographics2021.csv', dtype={'Zip Code': 'str'})
    zipInfodf = demographicsdf[demographicsdf['Zip Code'] == zipCode]
    return zipInfodf

In [16]:
# helper method to format json response
def formatZipData(zipInfodf):
    zipCols = zipInfodf.columns
    zipValues = zipInfodf.values[0]
    #zipValues = [zipValues[0]] + [round(3.13*val) for val in zipValues[1:]]
    zipInfodf = zipInfodf.T.set_axis(['Info'], axis=1)
    zipInfodf['Info'] = zipValues
    zipInfo = {'Zip Code': zipValues[0], 'Population' : zipValues[1]}
    zipInfo.update(zipInfodf[2:9].set_axis(['Race'], axis=1).to_dict())
    zipInfo.update(zipInfodf[9:11].set_axis(['Hispanic'], axis=1).to_dict())
    zipInfo.update(zipInfodf[11:18].set_axis(['Age'], axis=1).to_dict())
    zipInfo.update(zipInfodf[18:].set_axis(['Education'], axis=1).to_dict())
    return zipInfo

In [17]:
# Test result
demographics('91007')

{'Zip Code': '91007',
 'Population': 11708,
 'Race': {'White': 3867,
  'Black or African American': 218,
  'American Indian and Alaska Native': 30,
  'Asian': 6458,
  'Native Hawaiian and Other Pacific Islander': 40,
  'Some other race': 623,
  'Two or more races': 472},
 'Hispanic': {'Hispanic or Latino origin': 1692,
  'White alone, not Hispanic or Latino': 3201},
 'Age': {'Under 35 years': 1474,
  '35 to 44 years': 2105,
  '45 to 54 years': 2677,
  '55 to 64 years': 2273,
  '65 to 74 years': 1619,
  '75 to 84 years': 775,
  '85 years and over': 785},
 'Education': {'Less than high school graduate': 745,
  'High school graduate (includes equivalency)': 1809,
  "Some college or associate's degree": 2269,
  "Bachelor's degree or higher": 6885}}